# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas 

In [1]:
import pandas as pd 
import numpy as np  

import matplotlib            
import matplotlib.pyplot as plt

import seaborn as sns        
color = sns.color_palette()
sns.set_style('darkgrid')

import os

from random import randint
from scipy.stats import randint as sp_randint

from time import time
from datetime import datetime

import sklearn               
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2

from warnings import simplefilter

%matplotlib inline
simplefilter(action='ignore', category=FutureWarning)

### Carregamento dos Dados

In [2]:
treino = pd.read_csv("../../../../Dados/dados_treino.csv")
teste = pd.read_csv("../../../../Dados/dados_teste.csv")

### Preparação dos Dados

O primeiro passo é a preparação dos dados para o tipo de rede que se está a criar. Assim, tendo em conta que será utilizado o método *RandomizedSearchCV*, este apenas recebe um *array* de *features* e outro de *labels*, ambos unidimensionais, não pode ser efetuado o *reshape* dos dados neste passo.

In [3]:
from keras.utils.np_utils import to_categorical
Y = treino['LABEL']
X = treino.loc[:, treino.columns != 'LABEL']
x_train = X.values#.reshape(X.shape[0], 1, X.shape[1])

# One Hot Encoding
y_train = to_categorical(Y.values)
y_train = y_train[:, 1:]

Using TensorFlow backend.


As mesmas transformações têm que ser aplicadas aos dados do conjunto de teste.

In [4]:
Y = teste['LABEL']
X = teste.loc[:, teste.columns != 'LABEL']
x_test = X.values#.reshape(X.shape[0], 1, X.shape[1])

# One Hot Encoding
y_test = to_categorical(Y.values)
y_test = y_test[:, 1:]

### Modelo

Tendo em conta a utilização do método *RandomizedSearchCV*, foi necessária a definição de uma função para criar e retornar o modelo, com base em certos parâmetros, nomeadamente a taxa de aprendizagem, o número de neurónios e a probabilidade de *dropout*. Além disso, o modelo difere do original na medida em que possui uma camada inicial de *reshape* dos dados, devido às restrições no formato de dados do método de pesquisa dos parâmetros.

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Reshape
from keras.optimizers import Adam

def  create_model(learn_rate = 0.001, hidden = 848,dropout = 0):
    model = Sequential()
    model.add(Reshape((1, 3198)))
    # Unidades de Memória
    model.add(LSTM(hidden, input_shape=(1, 3198), dropout=dropout))
    # 2 Nodos de Saída
    model.add(Dense(2, activation='softmax'))
    
    adam = Adam(lr=learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

Para criar um modelo, é necessário instanciar uma variável como um *KerasClassifier*, sendo construído com base na função definida anteriormente para gerar um modelo.

In [6]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose = 1)

De forma a otimizar os parâmetros do modelo, estes devem ser definidos na forma de dicionário. Neste caso, optou-se pelo teste de várias combinações, vendo qual a que melhores resultados traria ao problema. Assim, foram testados os impactos de parâmetros como *batch_size*, número de épocas, taxa de aprendizagem, número de neurónios e probabilidade de *dropout*.

In [20]:
parametros = {
    'batch_size': [64, 128, 256],
    'epochs': [100, 200, 250, 300],
    'learn_rate': [0.001, 0.01, 0.0005, 0.0001, 0.00005, 0.00001],
    'hidden' : [64, 128, 256, 512, 848, 1024],
    'dropout' : [0.0, 0.1, 0.2, 0.25]
}

De modo a testar cada caso na grelha, foi definida uma métrica personalizada, permitindo comparar os valores reais com os calculados, retornando a *accuracy* dos cálculos.

In [21]:
from sklearn.metrics import accuracy_score
def custom_metric(y_true, y_predicted):
    return accuracy_score(y_true.argmax(axis=-1), y_predicted)

Além da definição da função para a métrica, é, ainda, fundamental definir como *scorer* a função e qual a orientação ideal para os resultados, ou seja, se quanto mais elevado melhor, ou o oposto. Neste caso, quanto mais elevada a *accuracy*, melhor o modelo testado.

In [22]:
from sklearn.metrics import make_scorer
custom_score = make_scorer(custom_metric, greater_is_better=True)

Como mencionado anteriormente, foi tirado proveito do método *RandomizedSearchCV*, permitindo encontrar, de forma mais rápida, uma combinação bastante próxima da ideal, ou até mesmo a ideal, dentro das combinações de parâmetros fornecidas, quando em comparação com uma pesquisa em grelha. Assim, optou-se por uma validação cruzada de 3 subconjuntos, devido ao elevado peso computacional dos modelos, sendo utilizada como métrica de *scoring* a função definida em cima.

In [23]:
from sklearn.model_selection import RandomizedSearchCV
grid = RandomizedSearchCV(estimator=model, param_distributions=parametros, n_jobs=4, cv=3, n_iter=10, scoring=custom_score, verbose=2)

Tendo a grelha definida e pronta a efetuar a procura, apenas é necessário aplicar o método *fit* com o conjunto de dados de treino. Não faria sentido aplicar a validação cruzada aos dados de teste juntamente com os de treino, já que o modelo ficaria demasiado ajustado aos dados, perdendo a capacidade de classificar corretamente novos casos.

In [25]:
grid_result = grid.fit(x_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\jonyp\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed: 303.8min finished


Epoch 1/300
6565/6565 [==============================] - 8s 1ms/step - loss: 0.4994 - accuracy: 0.7117
Epoch 2/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.3464 - accuracy: 0.8393
Epoch 3/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.2910 - accuracy: 0.9051
Epoch 4/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.2397 - accuracy: 0.9298
Epoch 5/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.2103 - accuracy: 0.9363
Epoch 6/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.1890 - accuracy: 0.9439
Epoch 7/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.1700 - accuracy: 0.9520
Epoch 8/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.1597 - accuracy: 0.9496
Epoch 9/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.1462 - accuracy: 0.9575
Epoch 10/300
6565/6565 [==============================] - 7s 1ms/step - l

6565/6565 [==============================] - 7s 1ms/step - loss: 0.0057 - accuracy: 0.9994
Epoch 158/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0064 - accuracy: 0.9989
Epoch 159/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0078 - accuracy: 0.9982
Epoch 160/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0072 - accuracy: 0.9983
Epoch 161/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0061 - accuracy: 0.9994
Epoch 162/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0064 - accuracy: 0.9991
Epoch 163/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0071 - accuracy: 0.9982
Epoch 164/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0062 - accuracy: 0.9991
Epoch 165/300
6565/6565 [==============================] - 7s 1ms/step - loss: 0.0059 - accuracy: 0.9989
Epoch 166/300
6565/6565 [==============================] - 7s 1ms/ste

Utilizando o método *get_params* é possível observar os vários parâmetros associados à pesquisa efetuada.

In [26]:
grid_result.get_params()

{'cv': 3,
 'error_score': nan,
 'estimator__verbose': 1,
 'estimator__build_fn': <function __main__.create_model(learn_rate=0.001, hidden=848, dropout=0)>,
 'estimator': <keras.wrappers.scikit_learn.KerasClassifier at 0x2679a28fc08>,
 'iid': 'deprecated',
 'n_iter': 10,
 'n_jobs': 4,
 'param_distributions': {'batch_size': [64, 128, 256],
  'epochs': [100, 200, 250, 300],
  'learn_rate': [0.001, 0.01, 0.0005, 0.0001, 5e-05, 1e-05],
  'hidden': [64, 128, 256, 512, 848, 1024],
  'dropout': [0.0, 0.1, 0.2, 0.25]},
 'pre_dispatch': '2*n_jobs',
 'random_state': None,
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(custom_metric),
 'verbose': 2}

Como se pode ver de seguida, os melhores parâmetros de entre os testados são *batch_size* com tamanho 256, ou seja, a cada iteração, a rede é treinada com 256 registos. Além disso, o número de épocas que permitiu os melhores resultados foi 300. No que toca aos parâmetros da função de criação do modelo, o valor de *dropout* foi de 20%, querendo isto dizer que a cada fase existe uma probabilidade de 20% de cada neurónio ser desativado. Já a taxa de aprendizagem tomou o valor de 0.00001. Por fim, o número de neurónios da camada *LSTM* que juntamente com os restantes parâmetros trouxe melhores resultados foi de 1024.

In [27]:
grid_result.best_params_

{'learn_rate': 1e-05,
 'hidden': 1024,
 'epochs': 300,
 'dropout': 0.2,
 'batch_size': 256}

O melhor resultado obtido para a pesquisa em grelha, com os parâmetros acima mencionados, permitiu uma *accuracy* de sensivelmente 77.5%. Note-se que este valor está associado a uma validação cruzada com 3 subconjuntos de dados, representando uma média e estando extremamente dependente dos dados associados a cada partição para treino e consequente *performance* do modelo.

In [28]:
grid_result.best_score_

0.7745278661881788

Em seguida podem ser observados os vários resultados obtidos para as várias combinações de forma mais detalhada.

In [37]:
cv_res_df = pd.DataFrame(grid_result.cv_results_)
cv_res_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learn_rate,param_hidden,param_epochs,param_dropout,param_batch_size,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,833.492934,13.137183,0.545904,0.060287,0.01,64,300,0.1,64,"{'learn_rate': 0.01, 'hidden': 64, 'epochs': 3...",0.983097,1.000000,0.324497,0.769198,0.314527,5
1,5255.557231,147.956814,2.429315,0.076108,1e-05,1024,300,0.2,256,"{'learn_rate': 1e-05, 'hidden': 1024, 'epochs'...",0.999086,1.000000,0.324497,0.774528,0.318220,1
2,2265.896554,76.293252,1.060505,0.037131,5e-05,256,250,0,64,"{'learn_rate': 5e-05, 'hidden': 256, 'epochs':...",0.983097,0.998629,0.324497,0.768741,0.314192,7
3,695.742409,21.993752,0.790552,0.048869,0.0005,128,300,0.2,256,"{'learn_rate': 0.0005, 'hidden': 128, 'epochs'...",0.998173,0.998172,0.324497,0.773614,0.317573,3
4,689.236728,18.063134,0.984365,0.096624,0.0005,256,200,0,256,"{'learn_rate': 0.0005, 'hidden': 256, 'epochs'...",0.983097,0.994973,0.324497,0.767522,0.313304,9
5,1774.515682,26.875490,2.976396,0.238974,0.0005,1024,100,0.25,256,"{'learn_rate': 0.0005, 'hidden': 1024, 'epochs...",0.997259,0.998172,0.324497,0.773309,0.317358,4
6,1010.361044,42.261563,0.791550,0.189941,0.001,128,200,0,64,"{'learn_rate': 0.001, 'hidden': 128, 'epochs':...",0.983097,0.989945,0.324497,0.765847,0.312094,10
7,1414.124263,23.333393,0.704290,0.099994,0.01,128,250,0.1,64,"{'learn_rate': 0.01, 'hidden': 128, 'epochs': ...",0.983097,1.000000,0.324497,0.769198,0.314527,5
8,4606.731557,12.151071,2.610422,0.062505,5e-05,1024,100,0.1,64,"{'learn_rate': 5e-05, 'hidden': 1024, 'epochs'...",0.998630,1.000000,0.324497,0.774376,0.318113,2
9,3936.294722,815.735177,1.353204,0.371426,0.0005,1024,300,0,256,"{'learn_rate': 0.0005, 'hidden': 1024, 'epochs...",0.983097,0.995887,0.324497,0.767827,0.313525,8


### Teste

Conhecidos os melhores parâmetros para este modelo, é necessário ver de que forma se comporta na classificação do conjunto de teste.

In [29]:
preds = grid_result.predict(x_test)

570/570 [==============================] - 0s 578us/step


De modo a comparar os resultados, de forma mais fácil, com as *labels* reais, as previsões foram convertidas em registos categóricos.

In [31]:
preds = to_categorical(preds)

Observando as métricas obtidas pela predição dos registos de teste, é notório que a precisão da classe minoritária aumentou quando em comparação com o modelo original (o qual tinha valor de 50%). Além disso, o *recall* manteve-se, possuindo um valor de 60% em ambos os casos.

In [33]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_test, preds))
print("accuracy:", accuracy_score(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       565
           1       0.60      0.60      0.60         5

   micro avg       0.99      0.99      0.99       570
   macro avg       0.80      0.80      0.80       570
weighted avg       0.99      0.99      0.99       570
 samples avg       0.99      0.99      0.99       570

accuracy: 0.9929824561403509


Observando as matrizes de confusão e, mais uma vez, comparando com o modelo original, verifica-se que foram classificados 3 dos 5 sistemas da classe minoritária corretamente, em ambos os casos. No que toca a classificações erradas de sistemas da classe maioritária, inicialmente foram incorretamente classificados 3 registos, tendo esse valor diminuído para apenas 2 sistemas com a otimização.

In [34]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, preds)

array([[[  3,   2],
        [  2, 563]],

       [[563,   2],
        [  2,   3]]], dtype=int64)

Em jeito de conclusão, a otimização do modelo não melhorou a capacidade do modelo em classificar corretamente os sistemas da classe minoritária no conjunto de teste. No entanto, tendo em conta a quantidade reduzida de registos da classe minoritária, os resultados obtidos são bastante bons, tendo sido reduzido o número de registos incorretamente classificados com esta otimização.